In [1]:
pip install transformers torch obsidian-vault


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement obsidian-vault (from versions: none)
ERROR: No matching distribution found for obsidian-vault


In [79]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [80]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
from collections import defaultdict
from tabulate import tabulate

# Model

https://huggingface.co/openai-community/gpt2

In [3]:
# Use a pipeline as a high-level helper
# pipe = pipeline("text-generation", model="openai-community/gpt2-large")

In [4]:
# Load pre-trained model and tokenizer
model_name = 'openai-community/gpt2-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add padding token to tokenizer
if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Load model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Vault

In [73]:
# Path to your Obsidian vault
vault_path = "../../Dokumenter/All my AI things/"

# Functions

In [55]:
def generate_text(prompt, max_length=1000):
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    attention_mask = inputs.attention_mask
    outputs = model.generate(inputs.input_ids, attention_mask=attention_mask, max_length=max_length, pad_token_id=tokenizer.eos_token_id, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def find_notes_with_content(query, vault_path):
    results = []
    found = False

    # Walk through the directory and find notes containing the query in the content
    for root, dirs, files in os.walk(vault_path):
        if found:
            break  # Stop searching if query is found
        for file in files:
            if file.endswith(".md"):  # Consider only markdown files
                filename = os.path.join(root, file)
                with open(filename, 'r', encoding='utf-8') as f:
                    content = f.read()
                    if query in content:  # Check if query is in the content
                        lines = content.splitlines()
                        for i, line in enumerate(lines):
                            if query in line:
                                result = (filename, i+1, line)
                                if result not in results:  # Avoid adding duplicate results
                                    results.append(result)
                        found = True  # Mark query as found

    # Sort results alphabetically by filename
    results.sort(key=lambda x: x[0])
    
    return results

def read_markdown_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return file.read()

def write_markdown_file(filename, content):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(content)

def get_file_statistics(vault_path):
    total_files = 0
    file_types_count = defaultdict(int)
    total_size_bytes = 0
    file_details = []  # List to store details for tabulation

    # Walk through the directory and gather statistics
    for root, dirs, files in os.walk(vault_path):
        for file in files:
            total_files += 1
            file_path = os.path.join(root, file)
            file_extension = os.path.splitext(file)[1].lower()

            # Count each file type
            file_types_count[file_extension] += 1

            # Calculate total size
            file_size_bytes = os.path.getsize(file_path)
            total_size_bytes += file_size_bytes

            # Append file details for tabulation
            file_details.append((file, file_extension, file_size_bytes))

    # Convert total size to human-readable format
    total_size_kb = total_size_bytes / 1024
    total_size_mb = total_size_kb / 1024
    total_size_gb = total_size_mb / 1024

    # Prepare statistics output
    statistics = {
        "Total Files": total_files,
        "File Types Count": dict(file_types_count),
        "Total Size (Bytes)": total_size_bytes,
        "Total Size (KB)": total_size_kb,
        "Total Size (MB)": total_size_mb,
        "Total Size (GB)": total_size_gb
    }

    return statistics, file_details

# Execution

## Vault stats

In [81]:
# Print some statistics about my vault.
statistics, file_details = get_file_statistics(vault_path)

In [82]:
# Print out the statistics
print("File Statistics:")
print(f"Total Files: {statistics['Total Files']}")

print("\nFile Types Count:")
for ext, count in statistics['File Types Count'].items():
    print(f"{ext}: {count}")

print("\nTotal Size:")
print(f"Bytes: {statistics['Total Size (Bytes)']}")
print(f"KB   : {statistics['Total Size (KB)']:.2f}")
print(f"MB   : {statistics['Total Size (MB)']:.2f}")
print(f"GB   : {statistics['Total Size (GB)']:.2f}")

# Format file details using tabulate
print("\nFile Details:")
print(tabulate(file_details, headers=['Filename', 'Filetype', 'Filesize (Bytes)'], tablefmt='grid'))

File Statistics:
Total Files: 20

File Types Count:
.jpg: 5
.md: 3
.json: 9
.js: 1
.css: 1
.canvas: 1

Total Size:
Bytes: 257098
KB   : 251.07
MB   : 0.25
GB   : 0.00

File Details:
+--------------------------------------------+------------+--------------------+
| Filename                                   | Filetype   |   Filesize (Bytes) |
+============================================+============+====================+
| rtyjtyjrty 2.jpg                           | .jpg       |              36640 |
+--------------------------------------------+------------+--------------------+
| rtyjtyjrty.jpg                             | .jpg       |              36640 |
+--------------------------------------------+------------+--------------------+
| thrthrth.jpg                               | .jpg       |              39064 |
+--------------------------------------------+------------+--------------------+
| thrthrthrt.jpg                             | .jpg       |              42837 |
+-------

## Find a word or phrase in a note

In [57]:
# Find something in the vault.
query = "Deep Learning"

results = find_notes_with_content(query, vault_path)

In [58]:

if results:
    num_results = len(results)
    print(f"Number of results: {num_results}")

    # Display results
    print("Results:")
    for i, result in enumerate(results):
        filename, row_number, line_content = result
        print(f"{i+1}. Filename: {filename}, Row Number: {row_number}, Line Content: {line_content}")

    # Process and update each matching note
    for result in results:
        filename, _, _ = result
        content = read_markdown_file(filename)
        updated_content = generate_text(content)
        write_markdown_file(filename, updated_content)
else:
    print(f"No notes containing '{query}' found in the vault.")

Number of results: 2
Results:
1. Filename: ../../Dokumenter/All my AI things/learning\Course Building Your Own Database Agent.md, Row Number: 6, Line Content: - Deep Learning for Data Science
2. Filename: ../../Dokumenter/All my AI things/learning\Course Building Your Own Database Agent.md, Row Number: 8, Line Content: - Deep Learning is cool.


KeyboardInterrupt: 